<a href="https://colab.research.google.com/github/Ieejuyeon/AlgorithmStudy/blob/FixPronounceKor/%EB%B0%9C%EC%9D%8C%EC%98%A4%EB%A5%98%EB%B3%80%ED%99%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install --upgrade transformers
import transformers
print(transformers.__version__)

4.51.3


In [10]:
import zipfile
import os

# Google Drive 내 ZIP 파일 경로
zip_file = 'drive/MyDrive/2025인공지능6팀/LcmsExcelView (1).zip'

# 압축을 풀 디렉토리 설정
extract_dir = 'drive/MyDrive/2025인공지능6팀/Lcms'

# 디렉토리가 없으면 생성
os.makedirs(extract_dir, exist_ok=True)

# ZIP 파일 해제
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# 압축 해제된 파일 목록 확인
extracted_files = os.listdir(extract_dir)
print("압축 해제된 파일들:", extracted_files)


압축 해제된 파일들: ['LcmsExcelView (1).xlsx', 'LcmsExcelView (2).xlsx', 'LcmsExcelView (3).xlsx', 'LcmsExcelView (4).xlsx', 'LcmsExcelView (5).xlsx', 'LcmsExcelView (6).xlsx', 'LcmsExcelView (7).xlsx', 'LcmsExcelView (8).xlsx', 'LcmsExcelView (9).xlsx', 'LcmsExcelView (10).xlsx', 'LcmsExcelView (11).xlsx', 'LcmsExcelView (12).xlsx', 'LcmsExcelView (13).xlsx', 'LcmsExcelView (14).xlsx', 'LcmsExcelView (15).xlsx', 'LcmsExcelView (16).xlsx', 'LcmsExcelView (17).xlsx', 'LcmsExcelView (18).xlsx', 'LcmsExcelView (19).xlsx', 'LcmsExcelView (20).xlsx', 'LcmsExcelView (21).xlsx', 'LcmsExcelView (22).xlsx', 'LcmsExcelView (23).xlsx', 'LcmsExcelView (24).xlsx', 'LcmsExcelView (25).xlsx', 'LcmsExcelView (26).xlsx', 'LcmsExcelView (27).xlsx', 'LcmsExcelView (28).xlsx', 'LcmsExcelView (29).xlsx', 'LcmsExcelView (30).xlsx']


In [ ]:

import os
file_name = "drive/MyDrive/2025인공지능6팀/LcmsExcelView (1).zip"
output_dir = "drive/MyDrive/2025인공지능6팀/Lcms"
os.system("unzip "+file_name+" -d "+output_dir)

데이터 준비

In [ ]:
import pandas as pd
import transformers
from transformers import Seq2SeqTrainingArguments

from datasets import Dataset
from transformers import (
    PreTrainedTokenizerFast,
    BartForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)
import torch

# 1. 모델과 토크나이저 로딩
model_name = "hyunwoongko/kobart"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# 2. 데이터셋 로딩
dfs = []


for i in range(1, 31):
    file_name = f"drive/MyDrive/2025인공지능6팀/Lcms/LcmsExcelView ({i}).xlsx"
    tempdf = pd.read_excel(file_name, header=2, usecols=['원 형태', '교정 형태'])
    dfs.append(tempdf)
df = pd.concat(dfs, ignore_index=True)
df["원 형태"] = df["원 형태"].astype(str)
df["교정 형태"] = df["교정 형태"].astype(str)
dataset = Dataset.from_pandas(df.reset_index(drop=True))

# 3. 전처리 함수 정의
max_length = 64

print(df)

def preprocess(example):
    input_enc = tokenizer(text=example["원 형태"], truncation=True, padding="max_length", max_length=max_length)
    target_enc = tokenizer(text_target=example["교정 형태"], truncation=True, padding="max_length", max_length=max_length)

    input_enc["labels"] = target_enc["input_ids"]
    return input_enc

tokenized_dataset = dataset.map(preprocess)

# 4. 데이터 분할
split_dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# 5. 트레이닝 설정
training_args = Seq2SeqTrainingArguments(
    output_dir="./kobart-corrector",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),  # GPU 사용 시 FP16
)

# 6. 트레이너 준비
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# 7. 학습 시작
trainer.train()
trainer.save_model("./kobart-corrector")  # 수동 저장
tokenizer.save_pretrained("./kobart-corrector")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default s

     원 형태 교정 형태
0       고     것
1       돔     좀
2     그런나   그러나
3      그로    그렇
4       께     게
...   ...   ...
2995    단     달
2996   단끼    딸기
2997    른     를
2998    단     달
2999  단연히   당연히

[3000 rows x 2 columns]


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

<ipython-input-11-dac60bb7378a>:69: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

In [ ]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

# 1. 모델과 토크나이저 로드 (경로는 학습된 모델 위치 또는 허깅페이스 경로)
model = BartForConditionalGeneration.from_pretrained("./kobart-corrector")
tokenizer = PreTrainedTokenizerFast.from_pretrained("./kobart-corrector")

# 2. 테스트 입력 문장
input_text = "목건이"

# 3. 토크나이즈 및 인코딩
inputs = tokenizer(input_text, return_tensors="pt")

# 4. 모델 예측 (디코딩)
output_ids = model.generate(**inputs, max_length=32)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("입력:", input_text)
print("출력:", output_text)
